In [34]:
from visdom import Visdom

class VisdomLinePlotter(object):
    """Plots to Visdom"""
    def __init__(self, env_name='main'):
        self.viz = Visdom()
        self.env = env_name
        self.plots = {}
    def plot(self, var_name, split_name, title_name, x, y):
        if var_name not in self.plots:
            self.plots[var_name] = self.viz.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
                legend=[split_name],
                title=title_name,
                xlabel='Epochs',
                ylabel=var_name
            ))
        else:
            self.viz.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[var_name], name=split_name, update = 'append')
            
            
    
vis = VisdomLinePlotter()

Setting up a new session...


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

Using TensorFlow backend.


In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
# device="cpu"

'Tesla V100-PCIE-16GB'

In [5]:
import numpy as np
import math
import pandas as pd

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import *

from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [6]:
df = pd.read_csv('reddit_train.csv')
df2 = pd.read_csv('reddit_test.csv')
# df = df.sample(1000, random_state=1).copy()
# df2 = df2.sample(1000, random_state=1).copy()
df.head()

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny


In [40]:
df['category_id'], mapping = df['subreddits'].factorize()

In [41]:
# Create sentence and label lists
sentences = df.comments.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.category_id.values

In [7]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

In [43]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁honestly', ',', '▁', 'buffalo', '▁is', '▁the', '▁correct', '▁answer', '.', '▁', 'i', '▁remember', '▁people', '▁', '(', 'some', 'what', ')', '▁joking', '▁that', '▁', 'buffalo', "'", 's', '▁man', 'tra', '▁for', '▁starting', '▁goalie', 's', '▁was', '▁', '"', 'win', '▁a', '▁game', ',', '▁get', '▁traded', '"', '.', '▁', 'i', '▁think', '▁', 'ed', 'mont', 'on', "'", 's', '▁front', '▁office', '▁was', '▁a', '▁', 'tra', 've', 's', 'ty', '▁for', '▁the', '▁better', '▁part', '▁of', '▁10', '▁years', ',', '▁but', '▁', 'buffalo', "'", 's', '▁systematic', '▁destruction', '▁of', '▁the', '▁term', '▁', "'", 'competitive', "'", '▁was', '▁much', '▁more', '▁responsible', '▁for', '▁the', '▁change', '▁to', '▁the', '▁draft', '▁lottery', '.', '▁[', 's', 'ep', ']', '▁[', 'cl', 's', ']']


In [44]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 512

In [45]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [46]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [47]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [48]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.05)

In [49]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [50]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 8

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [53]:
# Put model in evaluation mode to evaluate loss on the validation set
        model.eval()

        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = output[0]
            
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1

        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Validation Accuracy: 0.5633561643835616


In [8]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=20)
model.cuda()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [19]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]



In [20]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [52]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [23]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
step = 0

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    
    
    # Training
    
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    
    # Train the data for one epoch
    for batch in tqdm(train_dataloader,desc='batch',leave=False):
        step = step + 1
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        train_loss_set.append(loss.item()) 
        vis.plot('loss', 'train_loss', 'Loss',step,loss.item())   
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        
        
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
        
        
    # Validation

    if(step%500==0):

        # Put model in evaluation mode to evaluate loss on the validation set
        model.eval()

        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = output[0]
            
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1

        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
        vis.plot('accuracy', 'val_acc', 'val_acc',step,eval_accuracy/nb_eval_steps)
        torch.save(model, 'random_model.pt')

batch:   0%|          | 0/119 [00:00<?, ?it/s]

Train loss: 3.0547075572134066



batch:   0%|          | 0/119 [00:00<?, ?it/s]

Train loss: 2.8485440526689803



batch:  79%|███████▉  | 94/119 [01:29<00:23,  1.05it/s]

KeyboardInterrupt: 

In [31]:
df2=df

In [32]:
MAX_LEN=512

In [33]:
df2.comments.values.shape

(70000,)

In [34]:
# df2 = df2.iloc[:1000]

In [35]:
for i in range(df2.comments.values.shape[0]):
    df2.comments.values[i] = df2.comments.values[i][:512]

In [36]:
# Create sentence and label lists
sentences = df2.comments.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

In [37]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 



In [38]:
input_ids.shape

(70000, 512)

In [39]:
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
# prediction_labels = torch.tensor(labels)
  
batch_size = 32


prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [40]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []

# Predict 
for batch in tqdm(prediction_dataloader,desc='batch',leave=False):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = output[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()

    # Store predictions and true labels
    predictions.append(logits)

In [43]:
import pickle
pickle.dump( predictions, open( "preds/xlnet-preds-train.p", "wb" ) )

In [41]:
main_preds = []
for i in range(len(predictions)):
    main_preds += list(np.argmax(predictions[i], axis=1))
    
print(len(main_preds))

70000


In [42]:
main_preds

[0,
 1,
 7,
 3,
 11,
 4,
 5,
 6,
 5,
 7,
 11,
 16,
 6,
 7,
 8,
 8,
 6,
 3,
 3,
 7,
 8,
 11,
 3,
 6,
 2,
 12,
 14,
 3,
 9,
 13,
 18,
 14,
 11,
 3,
 13,
 17,
 18,
 3,
 17,
 2,
 15,
 7,
 17,
 12,
 17,
 16,
 18,
 17,
 0,
 6,
 4,
 6,
 8,
 1,
 8,
 5,
 10,
 11,
 15,
 11,
 11,
 3,
 0,
 10,
 5,
 10,
 19,
 9,
 13,
 0,
 5,
 6,
 0,
 12,
 9,
 6,
 7,
 3,
 4,
 3,
 18,
 9,
 9,
 1,
 6,
 6,
 17,
 6,
 18,
 12,
 0,
 13,
 6,
 11,
 16,
 17,
 2,
 12,
 1,
 13,
 15,
 4,
 11,
 7,
 12,
 9,
 19,
 4,
 15,
 13,
 12,
 13,
 11,
 0,
 14,
 5,
 1,
 3,
 2,
 4,
 13,
 6,
 6,
 16,
 16,
 17,
 5,
 5,
 11,
 9,
 17,
 7,
 2,
 11,
 7,
 8,
 9,
 7,
 18,
 13,
 6,
 7,
 9,
 3,
 18,
 18,
 4,
 10,
 18,
 14,
 1,
 10,
 4,
 14,
 4,
 1,
 8,
 19,
 1,
 16,
 18,
 16,
 14,
 15,
 16,
 11,
 16,
 7,
 6,
 14,
 6,
 18,
 4,
 11,
 13,
 2,
 1,
 0,
 8,
 17,
 11,
 3,
 4,
 10,
 18,
 18,
 7,
 12,
 2,
 1,
 18,
 17,
 17,
 17,
 3,
 4,
 14,
 5,
 8,
 3,
 1,
 9,
 6,
 10,
 13,
 11,
 18,
 1,
 18,
 4,
 5,
 11,
 13,
 8,
 4,
 17,
 18,
 19,
 8,
 14,
 10,
 10,
 1,
 0,


In [27]:
test_preds = pd.DataFrame()
test_preds['Id'] = df2['id']
test_preds['Category'] = mapping[main_preds]

NameError: name 'mapping' is not defined

In [28]:
test_preds

,Id
0,0
1,1
2,2
3,3
4,4
...,...
29995,29995
29996,29996
29997,29997
29998,29998


In [48]:
test_preds.to_csv("test2.csv", index=False)
from IPython.display import FileLink, FileLinks
FileLink('test2.csv')

/network/home/penmetss/comp551/test2.csv

In [11]:
model.load_state_dict(a.state_dict())

<All keys matched successfully>

In [10]:
a = torch.load('models/xlnet_model_final_standardEp1.5_fulldata.pt')